## OpenAI Function Calling with Tool Usage

In [56]:
from openai import OpenAI
import json
from os import getenv

client = OpenAI(api_key=getenv("OPENAI_API_KEY"))

def schedule_meeting(date, time, attendees):
    # Connect to calendar service:
    return { "event_id": "1234", "status": "Meeting scheduled successfully!",
            "date": date, "time": time, "attendees": attendees }

OPENAI_FUNCTIONS = {
    "schedule_meeting": schedule_meeting
}

In [57]:
# Our predefined function JSON schema:
functions = [
    {
        "type": "function",
        "function": {
            "type": "object",
            "name": "schedule_meeting",
            "description": "Set a meeting at a specified date and time for designated attendees",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "format": "date"},
                    "time": {"type": "string", "format": "time"},
                    "attendees": {"type": "array", "items": {"type": "string"}},
                },
                "required": ["date", "time", "attendees"],
            },
        },
    }
]

In [62]:
# Start the conversation:
messages = [
    {
        "role": "user",
        "content": "Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob.",
    }
]

# Send the conversation and function schema to the model:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=functions,
    tool_choice="auto"
)

response = response.choices[0].message

# Check if the model wants to call our function:
if response.tool_calls:
    # Get the first function call:
    first_tool_call = response.tool_calls[0]

    # Find the function name and function args to call:
    function_name = first_tool_call.function.name
    function_args = json.loads(first_tool_call.function.arguments)
    print("This is the function name: ", function_name)
    print("These are the function arguments: ", function_args)

    function = OPENAI_FUNCTIONS.get(function_name)

    if not function:
        raise Exception(f"Function {function_name} not found.")

    # Call the function:
    function_response = function(**function_args)

    # Share the function's response with the model:
    messages.append(
        {
            "role": "function",
            "name": "schedule_meeting",
            "content": json.dumps(function_response),
        }
    )

    # Let the model generate a user-friendly response:
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613", messages=messages
    )

    print(second_response.choices[0].message.content)

This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
I have scheduled a meeting with Alice and Bob on November 1, 2023, at 14:00. The event ID is 1234.


--------------------------------

## Parallel Function Calling:

In [55]:
# Start the conversation:
messages = [
    {
        "role": "user",
        "content": '''Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob. 
        Then I want to schedule another meeting on 2023-11-02 at 15:00 with Charlie and Dave.'''
    }
]

# Send the conversation and function schema to the model:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=functions,
)

response = response.choices[0].message

# Check if the model wants to call our function:
if response.tool_calls:
    for tool_call in response.tool_calls:
        # Get the function name and arguments to call:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        print("This is the function name: ", function_name)
        print("These are the function arguments: ", function_args)

        function = OPENAI_FUNCTIONS.get(function_name)

        if not function:
            raise Exception(f"Function {function_name} not found.")

        # Call the function:
        function_response = function(**function_args)

        # Share the function's response with the model:
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": json.dumps(function_response),
            }
        )

    # Let the model generate a user-friendly response:
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613", messages=messages
    )

    print(second_response.choices[0].message.content)

This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-01', 'time': '14:00', 'attendees': ['Alice', 'Bob']}
This is the function name:  schedule_meeting
These are the function arguments:  {'date': '2023-11-02', 'time': '15:00', 'attendees': ['Charlie', 'Dave']}
Two meetings have been scheduled:
1. Meeting with Alice and Bob on 2023-11-01 at 14:00.
2. Meeting with Charlie and Dave on 2023-11-02 at 15:00.


---

## Extracting Data with LangChain

In [28]:
from typing import Optional

from langchain.chains.openai_functions import (
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.pydantic_v1 import BaseModel, Field

class Article(BaseModel):
    """Identifying key points and contrarian views in an article."""

    points: str = Field(..., description="Key points from the article")
    contrarian_points: Optional[str] = Field(None, description="Any contrarian points acknowledged in the article")
    author: Optional[str] = Field(None, description="Author of the article")

# If we pass in a model explicitly, we need to make sure it supports the OpenAI function-calling API.
llm = ChatOpenAI(model="gpt-4", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a proficient algorithm capable of extracting structured information."),
        ("human", "*Extract* key points and any contrarian positions from the provided input using the specified format: {input}"),
        ("human", "*Remember to* produce your answer in the correct format"),
    ]
)

chain = create_structured_output_chain(Article, llm, prompt, verbose=True)
chain.run("In the recent article titled 'AI adoption in industry', key points addressed include the growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI. However, the author, Dr. Jane Smith, acknowledges a contrarian view — that without stringent regulations, AI may pose high risks. The anonymity of AI decision-making and the potential for misuse were stressed.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a proficient algorithm capable of extracting structured information.
Human: *Extract* key points and any contrarian positions from the provided input using the specified format: In the recent article titled 'AI adoption in industry', key points addressed include the growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI. However, the author, Dr. Jane Smith, acknowledges a contrarian view — that without stringent regulations, AI may pose high risks. The anonymity of AI decision-making and the potential for misuse were stressed.
Human: *Remember to* produce your answer in the correct format

> Finished chain.


Article(points='The growing interest in AI in various sectors, the increase in AI research, and the need for responsible AI.', contrarian_points='Without stringent regulations, AI may pose high risks. The anonymity of AI decision-making and the potential for misuse were stressed.', author='Dr. Jane Smith')